This blog post contains a few thoughts and selected highlights from my reading of the ABBA ([source code](https://github.com/thumbtack/abba), [documentation](http://thumbtack.github.io/abba/demo/abba.html)) and ExpAn ([source code](https://github.com/zalando/expan), [documentation](https://expan.readthedocs.io/en/latest/), [tutorial](https://expan.readthedocs.io/en/latest/tutorial.html)) libraries for A/B test analysis. For in-depth analysis, see the code and documentation for each package.

---

#### ABBA

ABBA is a Python framework for analysing the results of binomial metrics.

Two lessons learnt here:
- The use of `collections.namedtuple` in python as a quick way of creating an immutable class. Dan Bader, in his book *Python Tricks*, recommends using namedtuples for more readable and debuggable code.
- Different methods for computing a confidence interval for a binomial proportion. Typically I've used the normal approximation in computing the CI for binomial proportions, but the developers of ABBA use the Agresti-Coull Interval. Other options are available - see [wikipedia article](https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval) and [an in-depth article](https://projecteuclid.org/euclid.ejs/1402927499). All of these intervals offer different continuous approximations to the binomial distribution.

---

#### ExPan

In comparison to ABBA, ExPan is a more comprehensive, sophisticated tool providing a framework for ingesting, verifying, analysing and outputting the statistical analysis of randomised control trials. Here are some of the features I enjoyed most: 

- **Subgroup Analysis**: 
    - The user can specify different segments (e.g. device, demographic segments) for analysis. 
        - Digging into the different behaviour of subgroups can form the basis of **personalisation**. 
    - Currently, in ExpAn, this methodology works by having a user select a subgroup, then performs a statistical analysis. I gather there are plans to use machine learning to automate the detection of interesting subgroups in the future.
    - If these segments are defined up-front and analysed using the `StatisticalTestSuite` object, appropriate correction methods (the user can choose between Bonferroni or Benjamini-Hochberg) are applied. If this segmentation is performed as a follow-up a lower p-value threshold e.g. p-value of 1 x 10-4 should be used - this can be passed as the `alpha` argument when calling the `analyze_statistical_test` method on an experiment object.
    
    
- **Outlier analysis**:
    - Outlier analysis is a useful tool to incorporate into an A/B analysis tool. Outliers can:
        - skew your results
        - sway the result of an experiment if a few large outliers find their way into a particular group        
        - decrease your statistical power due to the increased variance of your metrics
        - may not be representative of your general population
    - ExpAn provides tools to remove outliers from KPI. By default, the method removes the data that is at or above the 99th percentile. Another approach suggested by [Ronny Kohavi](https://www.quora.com/How-would-you-run-an-A-B-test-if-the-observations-are-extremely-right-skewed) is to cap values, rather than percentiles. 
    
    
- **Early stopping methodologies**: 
    - When analysing an instance of the experiment class, the user can choose from 4 statistical methods:
        - fixed horizon
        - group sequential
        - bayes factor
        - bayes precision
    - The first method is used once an experiment has run its course; the last three are early stopping methods used to alert and abort bad experiments. 
    - The group sequential method is a frequentist approach which makes use of the O'Brien and Fleming procedure. Under this procedure, if a result is deemed statistically significant early, a more stringest threshold (lower p-value) is used - this matches our intuition that a higher bar (but lower p-value ;)) is required if a conclusion is based on fewer samples. 
        - The O'Brien and Fleming procedure forms part of Bing's Experimentation Monitoring system, as described in the paper *Online Controlled Experiments at Large Scale*. 
    
    
- **The implementation of the Benjamini-Hochberg correction:**
    - In ExpAn, the `benjamini_hochberg` function returns a corrected alpha value, which is subsequently used to compute the confidence intervals. I had not seen this previously and can find no clear answer on the subject - it appears to be an area of ongoing research. 

